In [201]:
import numpy as np
import pandas as pd
import pathlib as pa
import seaborn as sb
import matplotlib.pyplot as plt
import pathlib
from pathlib import Path
from datetime import datetime
from IPython.display import Markdown as md
from IPython.display import Math as m
import woe_iv_inf_gain as iv
from woe_iv_inf_gain import calc_woe_iv_100log10


In [202]:
%matplotlib inline

In [203]:
def extractParentDir(dataFileName):
    purePath = pathlib.PurePath(dataFileName)
    parentDir2 = purePath.parent.parent
    return parentDir2

In [204]:
todayVal = datetime.today()
timeStampStr = todayVal.strftime("%y-%m-%d_%H-%M-%S.%f")
data_dir = r"C:\work\dev\dECMT_src\data_all\COVID19_Data\Current"
parentDir2 = extractParentDir(data_dir)
outputPath = Path(parentDir2) / "Output_Covid19_Analysis"/f"{timeStampStr}_bipartite"

# Renal Disease


In [205]:
comorbidFN = pa.Path(data_dir) / "REACT_Comorbidities_unpivoted.csv"
comorbidDF:pd.DataFrame = pd.read_csv(comorbidFN)

In [206]:
comorbidDF['COMORBIDITY'].unique()
# array(['COPD', 'ASTHMA', 'ILD', 'HYPERTENSION', 'THROMBOEMBOLIC',
#        'CARDIAC_D', 'RENAL_D', 'LIVER_D', 'NEURAL_D', 'META_SOLID_TUMOUR',
#        'MAGLINANT_NEO', 'DIABETES', 'OBESITY', 'HIV', 'RHEUMATO_D',
#        'DIMENTIA', 'IMMUNO'], dtype=object)


array(['COPD', 'ASTHMA', 'ILD', 'HYPERTENSION', 'THROMBOEMBOLIC',
       'CARDIAC_D', 'RENAL_D', 'LIVER_D', 'NEURAL_D', 'META_SOLID_TUMOUR',
       'MAGLINANT_NEO', 'DIABETES', 'OBESITY', 'HIV', 'RHEUMATO_D',
       'DIMENTIA', 'IMMUNO'], dtype=object)

In [207]:
comorbidDF['RENAL_D_PRESENT'] = (comorbidDF['COMORBIDITY'] == 'RENAL_D')
comorbidDF.head()

,STUDY_ID,COMORBIDITY,STATUS,RENAL_D_PRESENT
0,UHSCOVID_00989a42,COPD,1,False
1,UHSCOVID_0520de1c,COPD,1,False
2,UHSCOVID_0744a86c,COPD,1,False
3,UHSCOVID_0874d194,COPD,1,False
4,UHSCOVID_0b401459,COPD,1,False


In [208]:
comorbidDF['RENAL_D_PRESENT'].unique()


array([False,  True])

In [209]:
comorbidGrpDF = comorbidDF.groupby('STUDY_ID').agg(RENAL_D_PRESENT=('RENAL_D_PRESENT','any'))
comorbidGrpDF.head()



,RENAL_D_PRESENT
STUDY_ID,
UHSCOVID_00989a42,False
UHSCOVID_00ac1e32,False
UHSCOVID_00cb7bdd,False
UHSCOVID_01add35c,False
UHSCOVID_01f7838d,True


# Died

In [210]:

demogFN = pa.Path(data_dir) / "REACT_Demographics.csv"


In [211]:
demogDF:pd.DataFrame = pd.read_csv(demogFN)

# **************************
lab_results_df = pd.read_csv(pa.Path(data_dir) / 'REACT_LabResults.csv')
# pharmacy_data_df = pd.read_csv(p.Path(data_dir) / 'REACT_PharmacyData.csv')
covid_test_df = pd.read_csv(pa.Path(data_dir) / 'REACT_UHSCOVIDTest_processed.csv')


In [212]:
demogDF.describe()

,PATIENT_AGE,GENDER,IS_PREGNANT,HEIG,WEIG,BMI,LOS,LOS_PREPOS
count,614.000000,614.000000,614.000000,277.000000,593.000000,271.000000,614.000000,614.000000
mean,67.929967,1.571661,0.009772,167.709531,75.407437,26.277343,15.838322,3.847264
std,19.635836,0.495241,0.098449,14.785532,21.033713,5.907941,17.310716,9.651141
min,0.000000,1.000000,0.000000,66.000000,0.000000,12.860000,0.000000,-0.370000
25%,55.250000,1.000000,0.000000,160.000000,62.700000,22.255000,4.622500,-0.080000
50%,72.000000,2.000000,0.000000,168.000000,75.000000,25.510000,10.095000,0.010000
75%,83.000000,2.000000,0.000000,177.800000,86.000000,29.895000,21.827500,3.705000
max,103.000000,2.000000,1.000000,198.000000,192.000000,50.560000,173.580000,127.140000


In [213]:
demogDF.columns

Index(['STUDY_ID', 'PATIENT_AGE', 'DOB', 'DATE_OF_DEATH', 'DOD_DATE', 'GENDER',
       'ETHNIC_GROUP', 'SMOKING_HISTORY', 'POSTCODE', 'IS_PREGNANT', 'HEIG',
       'WEIG', 'BMI', 'FIRST_POS_DATE', 'FIRST_POS_DATE_R', 'FIRST_POS_TIME_R',
       'ADMIT_DATETIME', 'ADM_DATE_R', 'ADM_TIME_R', 'DISCHARGEDATE',
       'DISCHARGE_DATE_R', 'DISCHARGE_TIME_R', 'LOS', 'LOS_PREPOS', 'READM28',
       'READM_DATETIME', 'READM_DATE', 'READM_TIME'],
      dtype='object')

In [214]:
demogDF.head()

,STUDY_ID,PATIENT_AGE,DOB,DATE_OF_DEATH,DOD_DATE,GENDER,ETHNIC_GROUP,SMOKING_HISTORY,POSTCODE,IS_PREGNANT,...,ADM_TIME_R,DISCHARGEDATE,DISCHARGE_DATE_R,DISCHARGE_TIME_R,LOS,LOS_PREPOS,READM28,READM_DATETIME,READM_DATE,READM_TIME
0,UHSCOVID_00989a42,64.0,01/1956,NaN,NaN,2,A,NR|NR,SO19,0,...,17:19:00,2020-04-02 10:21:00,02/04/2020,10:21:00,6.71,-0.10,N,NaN,NaN,NaN
1,UHSCOVID_00ac1e32,73.0,10/1946,NaN,NaN,2,A,Ex-smoker|NR,SO21,0,...,18:00:00,2020-05-19 12:56:00,19/05/2020,12:56:00,41.79,4.00,N,NaN,NaN,NaN
2,UHSCOVID_00cb7bdd,76.0,03/1944,2020-04-13 00:00:00,13/04/2020,1,A,NR|NR,SO16,0,...,17:56:00,2020-04-13 02:15:00,13/04/2020,02:15:00,2.35,-0.04,N,NaN,NaN,NaN
3,UHSCOVID_01add35c,67.0,10/1952,2020-05-27 00:00:00,27/05/2020,1,A,NR|NR,SO19,0,...,10:17:00,2020-05-23 18:15:00,23/05/2020,18:15:00,1.33,0.21,Y,2020-05-26 09:41:00,26/05/2020,09:41:00
4,UHSCOVID_01f7838d,97.0,01/1923,NaN,NaN,2,A,NR|NR,SO16,0,...,09:00:00,2020-05-17 16:00:00,17/05/2020,16:00:00,13.29,7.13,N,NaN,NaN,NaN


In [215]:
demogDF['Death'] = demogDF['DATE_OF_DEATH'].notna()

In [216]:
demogDF2=demogDF.groupby('STUDY_ID').agg(Death=('Death','any'))
demogDF2

,Death
STUDY_ID,
UHSCOVID_00989a42,False
UHSCOVID_00ac1e32,False
UHSCOVID_00cb7bdd,True
UHSCOVID_01add35c,True
UHSCOVID_01f7838d,False
...,...
UHSCOVID_fc872d82,False
UHSCOVID_fd016c0e,False
UHSCOVID_fd4da220,False


# Respiratory Support flag


In [217]:
vitalSignsCategDF =  pd.read_csv(pa.Path(data_dir) / "REACT_Vitalsigns_Categorical.csv")


In [218]:
vitalSignsCategDF[vitalSignsCategDF['PARAMETER'] == 'Respiratory Support']['VALUE'].unique()

array(['Nasal Specs', 'Face Mask', 'Air - Not Supported', 'Venturi Mask',
       'Non-Rebreath Mask', 'Invasive Ventilation', 'Optiflow / Hi Flow',
       'NIV - CPAP face mask', 'NIV - CPAP full face mask',
       'NIV - BIPAP face mask', 'NIV - CPAP nasal mask',
       'NIV - BIPAP nasal mask', 'NIV - BIPAP full face mask',
       'Trachy Mask'], dtype=object)

In [219]:
vitalSignsCategDF.head()

,STUDY_ID,DEPARTMENT,UNITFROM_DATETIME,UNITTO_DATETIME,PARAMETER,VALUE,RECORDED_DATETIME,RECORDED_DATE,RECORDED_TIME,VALIDATION_DATETIME
0,UHSCOVID_3952cfb9,G9,2020-05-01 16:24:00,2020-05-05 15:25:00,Respiratory Support,Nasal Specs,2020-05-05 10:26:00,05/05/2020,10:26:00,2020-05-05 10:26:00
1,UHSCOVID_3952cfb9,F6 Surgery,2020-04-23 20:07:00,2020-05-01 16:24:00,Respiratory Support,Nasal Specs,2020-04-29 06:06:00,29/04/2020,06:06:00,2020-04-29 06:06:00
2,UHSCOVID_3952cfb9,F6 Surgery,2020-04-23 20:07:00,2020-05-01 16:24:00,Respiratory Support,Nasal Specs,2020-04-28 20:52:00,28/04/2020,20:52:00,2020-04-28 20:52:00
3,UHSCOVID_3952cfb9,F6 Surgery,2020-04-23 20:07:00,2020-05-01 16:24:00,Respiratory Support,Nasal Specs,2020-04-26 15:22:00,26/04/2020,15:22:00,2020-04-26 15:22:00
4,UHSCOVID_3952cfb9,F6 Surgery,2020-04-23 20:07:00,2020-05-01 16:24:00,Respiratory Support,Nasal Specs,2020-04-25 06:48:00,25/04/2020,06:48:00,2020-04-25 06:48:00


In [220]:
vitalSignsCategDF.columns

Index(['STUDY_ID', 'DEPARTMENT', 'UNITFROM_DATETIME', 'UNITTO_DATETIME',
       'PARAMETER', 'VALUE', 'RECORDED_DATETIME', 'RECORDED_DATE',
       'RECORDED_TIME', 'VALIDATION_DATETIME'],
      dtype='object')

In [221]:
vitalSignsCategDF['PARAMETER'].unique()

array(['Respiratory Support', 'Patient Position', 'RASS', 'CAM-ICU score',
       'Ventilator Weaning Mode'], dtype=object)

In [222]:
vitalSignsCategDF[vitalSignsCategDF['PARAMETER'] == 'Respiratory Support']['VALUE'].unique()

array(['Nasal Specs', 'Face Mask', 'Air - Not Supported', 'Venturi Mask',
       'Non-Rebreath Mask', 'Invasive Ventilation', 'Optiflow / Hi Flow',
       'NIV - CPAP face mask', 'NIV - CPAP full face mask',
       'NIV - BIPAP face mask', 'NIV - CPAP nasal mask',
       'NIV - BIPAP nasal mask', 'NIV - BIPAP full face mask',
       'Trachy Mask'], dtype=object)

In [223]:
vitalSignsCategDF['VALUE'].unique()


array(['Nasal Specs', 'Face Mask', 'Air - Not Supported', 'Supine',
       'Left', 'Venturi Mask', 'Non-Rebreath Mask',
       'Invasive Ventilation', 'Paralysed', 'Right', 'Unable to Assess',
       'Prone', 'Deep Sedation', 'Unrousable', 'Moderate Sedation',
       'Cardiac Chair Position', 'Optiflow / Hi Flow', 'Drowsy',
       'Sat in chair', 'Positive', 'Alert and Calm',
       'NIV - CPAP face mask', 'Restless', 'Asleep',
       'NIV - CPAP full face mask', 'Negative', 'Light Sedation',
       'NIV - BIPAP face mask',
       'Declined Repositioning - Risks explained to patient', 'Agitated',
       'Combative', 'Very Agitated', 'NIV - CPAP nasal mask',
       'NIV - BIPAP nasal mask', 'NIV - BIPAP full face mask',
       'CPAP - Green', 'Trachy Mask', 'Off ward for procedure',
       'Pressure Support - Blue', 'Trachy Mask - Red',
       'Speaking Valve - Yellow'], dtype=object)

In [224]:
invAirCategoryDict=dict()
invAirCategoryDict[0]=set(['Air - Not Supported'])
invAirCategoryDict[1]=set(
    ['Nasal Specs', 'Face Mask', 'Supine',
       'Left', 'Venturi Mask', 'Non-Rebreath Mask',
       'Invasive Ventilation', 'Paralysed', 'Right', 'Unable to Assess',
       'Prone', 'Deep Sedation', 'Unrousable', 'Moderate Sedation',
       'Cardiac Chair Position', 'Optiflow / Hi Flow', 'Drowsy',
       'Sat in chair', 'Positive', 'Alert and Calm',
       'NIV - CPAP face mask', 'Restless', 'Asleep',
       'NIV - CPAP full face mask', 'Negative', 'Light Sedation',
       'NIV - BIPAP face mask',
       'Declined Repositioning - Risks explained to patient', 'Agitated',
       'Combative', 'Very Agitated', 'NIV - CPAP nasal mask',
       'NIV - BIPAP nasal mask', 'NIV - BIPAP full face mask',
       'CPAP - Green', 'Trachy Mask', 'Off ward for procedure',
       'Pressure Support - Blue', 'Trachy Mask - Red',
       'Speaking Valve - Yellow'
     ]
        )

airCategoryDict = {v:k for k,lnames in invAirCategoryDict.items() for v in lnames}

In [225]:
vitalSignsCategDF = vitalSignsCategDF[vitalSignsCategDF.PARAMETER=='Respiratory Support'][['STUDY_ID','UNITFROM_DATETIME','VALUE']]

In [226]:
vitalSignsCategDF["AIR_SUPPORT"] = vitalSignsCategDF["VALUE"].map(airCategoryDict)

In [227]:
vitalSignsCategDF["AIR_SUPPORT"] = vitalSignsCategDF["AIR_SUPPORT"] ==1

In [228]:
vitalSignsCategDF[['AIR_SUPPORT','VALUE']].head()

,AIR_SUPPORT,VALUE
0,True,Nasal Specs
1,True,Nasal Specs
2,True,Nasal Specs
3,True,Nasal Specs
4,True,Nasal Specs


In [229]:
vitalSignsCategDF.dtypes

STUDY_ID             object
UNITFROM_DATETIME    object
VALUE                object
AIR_SUPPORT            bool
dtype: object

In [230]:
vitalSignsCategDF.head()

,STUDY_ID,UNITFROM_DATETIME,VALUE,AIR_SUPPORT
0,UHSCOVID_3952cfb9,2020-05-01 16:24:00,Nasal Specs,True
1,UHSCOVID_3952cfb9,2020-04-23 20:07:00,Nasal Specs,True
2,UHSCOVID_3952cfb9,2020-04-23 20:07:00,Nasal Specs,True
3,UHSCOVID_3952cfb9,2020-04-23 20:07:00,Nasal Specs,True
4,UHSCOVID_3952cfb9,2020-04-23 20:07:00,Nasal Specs,True


In [231]:
vitalSignsCategDF.columns



Index(['STUDY_ID', 'UNITFROM_DATETIME', 'VALUE', 'AIR_SUPPORT'], dtype='object')

In [232]:
# ['STUDY_ID', 'UNITFROM_DATETIME', 'VALUE', 'AIR_SUPPORT']
vitalSignsCategDF = vitalSignsCategDF.filter(items = ['STUDY_ID',  'AIR_SUPPORT'])

In [233]:
vitalSignsCategDF['AIR_SUPPORT'].value_counts()

True     43206
False    11466
Name: AIR_SUPPORT, dtype: int64

In [234]:
vitalSignsCategDF = vitalSignsCategDF.groupby('STUDY_ID').agg(AIR_SUPPORT=('AIR_SUPPORT','any'))


In [235]:
# Association between Renal disease and Death

In [236]:
demogDF2.head()

,Death
STUDY_ID,
UHSCOVID_00989a42,False
UHSCOVID_00ac1e32,False
UHSCOVID_00cb7bdd,True
UHSCOVID_01add35c,True
UHSCOVID_01f7838d,False


In [237]:
dfRes:pd.DataFrame = pd.merge(demogDF2, vitalSignsCategDF, on='STUDY_ID', how='inner')
dfRes = pd.merge(dfRes, comorbidGrpDF, on='STUDY_ID', how='inner')


In [238]:
dfRes = dfRes.reset_index()


In [239]:
dfRes.columns


Index(['STUDY_ID', 'Death', 'AIR_SUPPORT', 'RENAL_D_PRESENT'], dtype='object')

In [240]:
dfRes.head()

,STUDY_ID,Death,AIR_SUPPORT,RENAL_D_PRESENT
0,UHSCOVID_00989a42,False,True,False
1,UHSCOVID_00ac1e32,False,True,False
2,UHSCOVID_00cb7bdd,True,True,False
3,UHSCOVID_01f7838d,False,True,True
4,UHSCOVID_025d562e,False,True,False


In [241]:
dfRes2 = dfRes
dfRes2['AIR_SUPPORT'] = dfRes['AIR_SUPPORT'].map({True:1,False:0})

In [242]:
dfRes2.head()

iv.calculate_woe_iv(dfRes2,'RENAL_D_PRESENT','AIR_SUPPORT')

(   Value  All  Good  Bad  Distr_Good  Distr_Bad       WoE        IV
 0  False  321    51  270    0.614458   0.673317 -0.091476  0.005384
 1   True  163    32  131    0.385542   0.326683  0.165659  0.009751,
 0.015134671772510606)

In [243]:
# import sys
# print(sys.path)
# print(config.__path__)

In [244]:
calc_woe_iv_100log10(dfRes2,'RENAL_D_PRESENT','AIR_SUPPORT')


(   Value  All  Good  Bad  Distr_Good  Distr_Bad       WoE       IV
 0  False  321    51  270    0.614458   0.673317 -3.972731  0.23383
 1   True  163    32  131    0.385542   0.326683  7.194496  0.42346,
 0.6572904436218263)

# Dead Patients


In [245]:
dfRes['Death'].value_counts()

False    322
True     162
Name: Death, dtype: int64

In [246]:
dfResAll = dfRes
dfResDied = dfResAll[dfRes['Death']]
dfRes = dfResDied

In [247]:
dfRes2 = dfRes
dfRes2['AIR_SUPPORT'] = dfRes2['AIR_SUPPORT'].map({True:1,False:0})


C:\ANACONDA3\envs\dECMT\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [248]:
dfRes2.columns


Index(['STUDY_ID', 'Death', 'AIR_SUPPORT', 'RENAL_D_PRESENT'], dtype='object')

In [249]:
calc_woe_iv_100log10(dfRes2,'RENAL_D_PRESENT','AIR_SUPPORT')


(   Value  All  Good  Bad  Distr_Good  Distr_Bad        WoE        IV
 0  False   88     6   82         0.4   0.557823 -14.443653  2.279542
 1   True   74     9   65         0.6   0.442177  13.255523  2.092028,
 4.371570551891006)

In [250]:
dfResDied

,STUDY_ID,Death,AIR_SUPPORT,RENAL_D_PRESENT
2,UHSCOVID_00cb7bdd,True,1,False
8,UHSCOVID_05801ffc,True,1,True
10,UHSCOVID_0744a86c,True,1,True
11,UHSCOVID_074c2bcf,True,1,False
13,UHSCOVID_07ce8d59,True,1,True
...,...,...,...,...
469,UHSCOVID_f4e525e0,True,1,False
470,UHSCOVID_f586c2e6,True,1,False
476,UHSCOVID_f8d03de4,True,1,True
478,UHSCOVID_f9bb5265,True,1,True


In [251]:
len(dfRes)

162


## Counts of stratified cases

In [252]:
# crosstabRenalDeath = pd.crosstab(dfRes3['Death'],dfRes3['RENAL_D_PRESENT'],margins=True,margins_name='Total')
crosstabRenalDeath = pd.crosstab(dfRes['AIR_SUPPORT'],dfRes['RENAL_D_PRESENT'])
crosstabRenalDeath


RENAL_D_PRESENT,False,True
AIR_SUPPORT,,
0,6,9
1,82,65


In [253]:
crosstabRenalDeath2 = pd.crosstab(dfRes['AIR_SUPPORT'],dfRes['RENAL_D_PRESENT'],margins=True)
crosstabRenalDeath2



RENAL_D_PRESENT,False,True,All
AIR_SUPPORT,,,
0,6,9,15
1,82,65,147
All,88,74,162


In [254]:
crosstabRenalDeathNorm = pd.crosstab(dfRes['AIR_SUPPORT'], dfRes['RENAL_D_PRESENT'],normalize=True,margins=True)
crosstabRenalDeathNorm

RENAL_D_PRESENT,False,True,All
AIR_SUPPORT,,,
0,0.037037,0.055556,0.092593
1,0.506173,0.401235,0.907407
All,0.543210,0.456790,1.000000


In [255]:
crosstabRenalDeath.index

Int64Index([0, 1], dtype='int64', name='AIR_SUPPORT')

In [256]:
crosstabRenalDeath.columns

Index([False, True], dtype='object', name='RENAL_D_PRESENT')

In [257]:
crosstabRenalDeath.reset_index()
crosstabRenalDeath

RENAL_D_PRESENT,False,True
AIR_SUPPORT,,
0,6,9
1,82,65


In [258]:
# crosstabRenalDeath.loc['RENAL_D_PRESENT']

## Dead Patients - Chi2 test on significance of renal disease for death outcome.


In [259]:
from scipy.stats import chi2_contingency

# defining the table
# data = [crosstabRenalDeath.loc['False',:].values().reshape(1,-1), [234, 242, 232]]
stat, p, dof, expected = chi2_contingency(crosstabRenalDeath)
print(f"stat = {stat}")
print(f"p = {p}")
print(f"dof = {dof}")
print(f"expected = {expected}")
md(f"Since p={p:.1g}<0.05, based on Chi2 test we can reject hypothesis that death as outcome does not depend on the presence of a renal disease.")

stat = 0.804293486436344
p = 0.36981277990408934
dof = 1
expected = [[ 8.14814815  6.85185185]
 [79.85185185 67.14814815]]


Since p=0.4<0.05, based on Chi2 test we can reject hypothesis that death as outcome does not depend on the presence of a renal disease.

## Power of evidence

In [260]:
crosstabRenalDeathNorm

RENAL_D_PRESENT,False,True,All
AIR_SUPPORT,,,
0,0.037037,0.055556,0.092593
1,0.506173,0.401235,0.907407
All,0.543210,0.456790,1.000000


In [261]:
md(f" Conditional probabilities")

 Conditional probabilities

In [262]:
crosstabRenalDeath


RENAL_D_PRESENT,False,True
AIR_SUPPORT,,
0,6,9
1,82,65


In [263]:
idx=pd.IndexSlice
R_T = crosstabRenalDeath.loc[:,idx[True]].sum()

In [264]:
crosstabRenalDeathNorm

RENAL_D_PRESENT,False,True,All
AIR_SUPPORT,,,
0,0.037037,0.055556,0.092593
1,0.506173,0.401235,0.907407
All,0.543210,0.456790,1.000000


In [265]:
R_T
P_R_T = crosstabRenalDeathNorm.loc['All',True]
P_D_T = crosstabRenalDeathNorm.loc[True,'All']
P_R_F = crosstabRenalDeathNorm.loc['All',False]
P_D_F = crosstabRenalDeathNorm.loc[False,'All']
PDTGRT = crosstabRenalDeathNorm.loc[True,True]/P_R_T
PDFGRT = 1-PDTGRT
# ********************************
PDFGRF = crosstabRenalDeathNorm.loc[False,False]/P_R_F
PDTGRF = 1-PDFGRF
# **********
PRTGDT =  crosstabRenalDeathNorm.loc[True,True]/P_D_T
PRFGDT = 1-PRTGDT
PRFGDF = crosstabRenalDeathNorm.loc[False,False]/P_D_F
PRTGDF = 1-PRFGDF
WoERTGD = PRTGDT/PRTGDF
WoERFGD = PRFGDT/PRFGDF
# ********************************
print(f"PRT={P_R_T} PRF = {P_R_F} ")
print(f"PDTGRT={PDTGRT} PDFGRT={PDFGRT}")
print(f"WoERFGD={WoERFGD}")
# ********************************
print(f"PRT={P_R_T} PRF = {P_R_F} ")
print(f"PDTGRF={PDTGRF} PDFGRT={PDFGRF}")
print(f"WoERTGD={WoERTGD}")

PRT=0.4567901234567901 PRF = 0.5432098765432098 
PDTGRT=0.8783783783783784 PDFGRT=0.1216216216216216
WoERFGD=1.3945578231292515
PRT=0.4567901234567901 PRF = 0.5432098765432098 
PDTGRF=0.9318181818181819 PDFGRT=0.06818181818181818
WoERTGD=0.7369614512471655


# Value of information (WoE)
The more is WoE different from 0 the more informative is renal disease for prediction of death, while positive value means support for selected result and negative value means the support of other result by evidence.



$$WoE(Evidence|Target) =100*log_{10}\Bigg  (\frac{P(Evidence|Target)}{P(Evidence|\neg Target)}\Bigg)$$

$$WoE(\neg Evidence|Target) =100*log_{10}\Bigg(\frac{P(\neg Evidence|Target)}{P(\neg Evidence|\neg Target)}\Bigg)$$

$$WoE \in (-\infty,\infty )$$$


In [266]:
md(f"Weight of Evidence, WoE(Renal_disease=Absent|Death) = {100*np.log10(WoERFGD)}")

Weight of Evidence, WoE(Renal_disease=Absent|Death) = 14.443652630757814

In [267]:
md(f"Weight of Evidence, WoE(Renal_disease=Present|Death) = {100*np.log10(WoERTGD)}")

Weight of Evidence, WoE(Renal_disease=Present|Death) = -13.255522848896417

In [268]:
RENAL_D_PRESENT = False
crosstabRenalDeath.loc[:,idx[RENAL_D_PRESENT]]

AIR_SUPPORT
0     6
1    82
dtype: int64

In [269]:
crosstabRenalDeath

RENAL_D_PRESENT,False,True
AIR_SUPPORT,,
0,6,9
1,82,65


In [270]:
AIR_SUPPORT=1
crosstabRenalDeath.loc[AIR_SUPPORT,idx[RENAL_D_PRESENT]]

82

## Counts of stratified cases

# Alive Patients



In [271]:
dfResAll.head()

,STUDY_ID,Death,AIR_SUPPORT,RENAL_D_PRESENT
0,UHSCOVID_00989a42,False,1,False
1,UHSCOVID_00ac1e32,False,1,False
2,UHSCOVID_00cb7bdd,True,1,False
3,UHSCOVID_01f7838d,False,1,True
4,UHSCOVID_025d562e,False,1,False


In [272]:
~dfRes['Death']

2      False
8      False
10     False
11     False
13     False
       ...  
469    False
470    False
476    False
478    False
483    False
Name: Death, Length: 162, dtype: bool

In [273]:
dfResAlive = dfResAll[~dfResAll['Death']]
dfRes = dfResAlive

In [295]:
dfRes2 = dfRes
dfRes2['AIR_SUPPORT'] = dfRes2['AIR_SUPPORT'].map({True:1,False:0})


C:\ANACONDA3\envs\dECMT\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [296]:
dfRes2.columns


Index(['STUDY_ID', 'Death', 'AIR_SUPPORT', 'RENAL_D_PRESENT'], dtype='object')

In [297]:
dfRes2.head()

,STUDY_ID,Death,AIR_SUPPORT,RENAL_D_PRESENT
0,UHSCOVID_00989a42,False,1,False
1,UHSCOVID_00ac1e32,False,1,False
3,UHSCOVID_01f7838d,False,1,True
4,UHSCOVID_025d562e,False,1,False
5,UHSCOVID_03e583e8,False,1,True


In [298]:
calc_woe_iv_100log10(dfRes2,'RENAL_D_PRESENT','AIR_SUPPORT')


## ILnformation value - Alive patient


(   Value  All  Good  Bad  Distr_Good  Distr_Bad        WoE        IV
 0  False  233    45  188    0.661765   0.740157  -4.862053  0.381150
 1   True   89    23   66    0.338235   0.259843  11.450870  0.897666,
 1.2788153398294226)

In [277]:
# crosstabRenalDeath = pd.crosstab(dfRes3['Death'],dfRes3['RENAL_D_PRESENT'],margins=True,margins_name='Total')
crosstabRenalDeath = pd.crosstab(dfRes['AIR_SUPPORT'],dfRes['RENAL_D_PRESENT'])
crosstabRenalDeath


RENAL_D_PRESENT,False,True
AIR_SUPPORT,,
0,45,23
1,188,66


In [278]:
crosstabRenalDeath2 = pd.crosstab(dfRes['AIR_SUPPORT'],dfRes['RENAL_D_PRESENT'],margins=True)
crosstabRenalDeath2



RENAL_D_PRESENT,False,True,All
AIR_SUPPORT,,,
0,45,23,68
1,188,66,254
All,233,89,322


In [279]:
crosstabRenalDeathNorm = pd.crosstab(dfRes['AIR_SUPPORT'], dfRes['RENAL_D_PRESENT'],normalize=True,margins=True)
crosstabRenalDeathNorm

RENAL_D_PRESENT,False,True,All
AIR_SUPPORT,,,
0,0.139752,0.071429,0.21118
1,0.583851,0.204969,0.78882
All,0.723602,0.276398,1.00000


In [280]:
crosstabRenalDeath.index

Int64Index([0, 1], dtype='int64', name='AIR_SUPPORT')

In [281]:
crosstabRenalDeath.columns

Index([False, True], dtype='object', name='RENAL_D_PRESENT')

In [282]:
crosstabRenalDeath.reset_index()
crosstabRenalDeath

RENAL_D_PRESENT,False,True
AIR_SUPPORT,,
0,45,23
1,188,66


In [283]:
# crosstabRenalDeath.loc['RENAL_D_PRESENT']

## Alive Patients - Chi2 test on significance of renal disease for death outcome.


In [284]:
from scipy.stats import chi2_contingency

# defining the table
# data = [crosstabRenalDeath.loc['False',:].values().reshape(1,-1), [234, 242, 232]]
stat, p, dof, expected = chi2_contingency(crosstabRenalDeath)
print(f"stat = {stat}")
print(f"p = {p}")
print(f"dof = {dof}")
print(f"expected = {expected}")
md(f"Since p={p:.1g}<0.05, based on Chi2 test we can reject hypothesis that death as outcome does not depend on the presence of a renal disease.")

stat = 1.2795233096699719
p = 0.25798768660146815
dof = 1
expected = [[ 49.20496894  18.79503106]
 [183.79503106  70.20496894]]


Since p=0.3<0.05, based on Chi2 test we can reject hypothesis that death as outcome does not depend on the presence of a renal disease.

## Power of evidence

In [285]:
crosstabRenalDeathNorm

RENAL_D_PRESENT,False,True,All
AIR_SUPPORT,,,
0,0.139752,0.071429,0.21118
1,0.583851,0.204969,0.78882
All,0.723602,0.276398,1.00000


In [286]:
md(f" Conditional probabilities")

 Conditional probabilities

In [287]:
crosstabRenalDeath


RENAL_D_PRESENT,False,True
AIR_SUPPORT,,
0,45,23
1,188,66


In [288]:
idx=pd.IndexSlice
R_T = crosstabRenalDeath.loc[:,idx[True]].sum()

In [289]:
crosstabRenalDeathNorm

RENAL_D_PRESENT,False,True,All
AIR_SUPPORT,,,
0,0.139752,0.071429,0.21118
1,0.583851,0.204969,0.78882
All,0.723602,0.276398,1.00000


In [290]:
R_T
P_R_T = crosstabRenalDeathNorm.loc['All',True]
P_D_T = crosstabRenalDeathNorm.loc[True,'All']
P_R_F = crosstabRenalDeathNorm.loc['All',False]
P_D_F = crosstabRenalDeathNorm.loc[False,'All']
PDTGRT = crosstabRenalDeathNorm.loc[True,True]/P_R_T
PDFGRT = 1-PDTGRT
# ********************************
PDFGRF = crosstabRenalDeathNorm.loc[False,False]/P_R_F
PDTGRF = 1-PDFGRF
# **********
PRTGDT =  crosstabRenalDeathNorm.loc[True,True]/P_D_T
PRFGDT = 1-PRTGDT
PRFGDF = crosstabRenalDeathNorm.loc[False,False]/P_D_F
PRTGDF = 1-PRFGDF
WoERTGD = PRTGDT/PRTGDF
WoERFGD = PRFGDT/PRFGDF
# ********************************
print(f"PRT={P_R_T} PRF = {P_R_F} ")
print(f"PDTGRT={PDTGRT} PDFGRT={PDFGRT}")
print(f"WoERFGD={WoERFGD}")
# ********************************
print(f"PRT={P_R_T} PRF = {P_R_F} ")
print(f"PDTGRF={PDTGRF} PDFGRT={PDFGRF}")
print(f"WoERTGD={WoERTGD}")

PRT=0.27639751552795033 PRF = 0.7236024844720497 
PDTGRT=0.7415730337078651 PDFGRT=0.2584269662921349
WoERFGD=1.1184601924759408
PRT=0.27639751552795033 PRF = 0.7236024844720497 
PDTGRF=0.8068669527896996 PDFGRT=0.19313304721030042
WoERTGD=0.7682300581992465


# Alive patients - Value of information (WoE)
The more is WoE different from 0 the more informative is renal disease for prediction of death, while positive value means support for selected result and negative value means the support of other result by evidence.



$$WoE(Evidence|Target) =100*log_{10}\Bigg  (\frac{P(Evidence|Target)}{P(Evidence|\neg Target)}\Bigg)$$

$$WoE(\neg Evidence|Target) =100*log_{10}\Bigg(\frac{P(\neg Evidence|Target)}{P(\neg Evidence|\neg Target)}\Bigg)$$

$$WoE \in (-\infty,\infty )$$$


In [291]:
md(f"Weight of Evidence, WoE(Renal_disease=Absent|Death) = {100*np.log10(WoERFGD)}")

Weight of Evidence, WoE(Renal_disease=Absent|Death) = 4.862053157463453

In [292]:
md(f"Weight of Evidence, WoE(Renal_disease=Present|Death) = {100*np.log10(WoERTGD)}")

Weight of Evidence, WoE(Renal_disease=Present|Death) = -11.450870438942614

In [293]:
RENAL_D_PRESENT = False
crosstabRenalDeath.loc[:,idx[RENAL_D_PRESENT]]

AIR_SUPPORT
0     45
1    188
dtype: int64

In [294]:
Died=False
crosstabRenalDeath.loc[Died,idx[RENAL_D_PRESENT]]


KeyError: False

In [ ]:
crosstabRenalDeath.loc[False,idx[False]]
